In [1]:
#imports
import requests
import pandas as pd
from datetime import datetime

In [2]:
token = '85GaAh9i4LaVP9BRVhapXCPao'

In [3]:
desti_folder = './data_folder/taxico_data'

In [4]:
timestamp = datetime.now()
timestamp = timestamp.strftime("%Y-%m-%d-%H-%M-%S")

In [5]:
# some key info
taxi_api = 'wrvz-psew'
limit = 500000
i = 19
offset_counter = 0

pickle = True

# dict for logging
log_dict = {}

# base url for the request
baseurl = ("https://data.cityofchicago.org/resource/"
        f"{taxi_api}.json?")

while pickle:
    #set our params
    params = (
            #f"$where=trip_start_timestamp%20%3E%20%272020-12-31T23%3A59%3A59.000%27&"
            f"$order=trip_start_timestamp%20 DESC&"
            f"$limit={limit}&"
            f"$offset={offset_counter}&"
            f"$$app_token={token}")
    
    # make the request 
    taxico_r = requests.get(baseurl+params,timeout=5000)

    # convert to json
    taxico_data = taxico_r.json()

    # make it a df
    df = pd.json_normalize(taxico_data)

    # write df
    df.to_json(f'{desti_folder}/taxico_chunk_{i}.json',orient='records', lines=True)
    
    # log
    minilog = {}
    minilog['records'] = df.shape[0]
    minilog['chunk'] = i
    minilog['status'] = taxico_r.status_code
    minilog['offset'] = offset_counter
    log_dict[f'run_{i}'] = minilog
    log_df = pd.DataFrame.from_dict(log_dict,orient='index')
    log_df.to_csv(f'{desti_folder}/taxico_run_{timestamp}_log.csv')

    # increment the chunk count
    print(i)
    i += 1

    # increment the offset
    offset_counter += limit

    # check if we need to end the loop
    if taxico_r.status_code == 200:
        pickle = True
    else: 
        pickle = False
        print(taxico_r.status_code)



19
20
21
22
23
24


KeyboardInterrupt: 

In [ ]:
taxico_r

<Response [408]>